Проверить, принадлежит ли объект к какому-либо классу можно с помощью функции __isinstance__:

In [1]:
lul = 'stolb'
print(isinstance(lul,int))
print(isinstance(lul,str))

False
True


С помощью __issubclass__, можно проверять, наследует ли один класс от другого. Каждый класс в питоне - наследник класса object.

In [7]:
print(issubclass(int,object))

True


Класс объявляется с помощью киворда (ВНЕЗАПНО) __class__, все "магические" методы можно посмотреть с помощью функции __dir__

In [3]:
class ESR:
    pass
print(dir(ESR))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


Например, метод **__str__** позволяет изменить результат возвращаемый при вызове функции принт на экземпляр (instance) объекта: 

In [5]:
class Mark:
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return f"VOT ETO PRIKOL {self.name}!"

In [10]:
kurp = Mark('Trade')
print(kurp)

VOT ETO PRIKOL Trade!


Тут важно понимать, что это работает при прямом вызове print, а вот если мы вызываем например из листа, то питон использует не __str__, а __repr__, его, впрочем, тоже можно переопределить

In [13]:
class Mark:
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return f"VOT ETO PRIKOL {self.name}"
    def __repr__(self):
        return f"Drugoi Prikol: {self.name}"

lister = ['kurp','burp', 'durp']
res = [Mark(i) for i in lister]
print(res)
print(res[0])

[Drugoi Prikol: kurp, Drugoi Prikol: burp, Drugoi Prikol: durp]
VOT ETO PRIKOL kurp


Атрибут ____dict____ покажет мне все атрибуты объекта, к которому применяется

In [9]:
print(kurp.__dict__)
print(Mark.__dict__)

{'name': 'Trade'}
{'__module__': '__main__', '__init__': <function Mark.__init__ at 0x000002D6C81D7D08>, '__str__': <function Mark.__str__ at 0x000002D6C8280EA0>, '__dict__': <attribute '__dict__' of 'Mark' objects>, '__weakref__': <attribute '__weakref__' of 'Mark' objects>, '__doc__': None}


Метод __mro__ показывает в каком порядке питон идет по цепочке наследования, очень полезно когда у нас много сабклассов. Например, в примере ниже мы видим, что в случае с целым числом сначала при поиске какого-либо атрибута питон сначала смотрит в в классе int, а потом в классе object

In [10]:
int.__mro__

(int, object)

Метод __new__ позволяет создать т.н. конструктор инстансов. Когда мы создаем по дефолту создаем инстанс вызывается метод __init__, однако если мы определим метод __new__, то сначала будет вызываться __new__, и лишь когда он отработает будет вызван __init__. Это полезно, например, когда мы хотим контролировать кол-во имеющихся инствнсов. Пример:

In [30]:
class Mark:
    _instances=[]
    limit=3 #т.е. мы не хотим иметь больше 5 инстансов
    def __new__(cls,*args,**kwargs):
        print("i am here")
        if len(cls._instances)>=cls.limit:
            print(1)
            raise RuntimeError(f"Не шмогла я, не шмогла, лимит всего {cls.limit}")
        instance = super().__new__(cls)
        cls._instances.append(instance)
        return instance
    
    def __init__(self,name):
        print ("here is init babe")
        self.name = name

lister = ['kurp','burp', 'durp', 'lurp']
res=[]
for i in lister:
    res.append(Mark(i))
print((len(res[1]._instances)))    

i am here
here is init babe
i am here
here is init babe
i am here
here is init babe
i am here
1


RuntimeError: Не шмогла я, не шмогла, лимит всего 3

Методы можно вызывать из других методов (по идее, особенно логично, когда у тебя есть родительских класс с каким-нибудь общим методом)

In [31]:
class Human:
    def __init__(self, name, age=0):
        self._name = name
        self._age = age
 
    def _say(self, text):   #имя с нижним подчервкиванием - конвенция по тому, как именовать методы,
                            #которые не предназначены к вызову напрямую, а нужны только для вызовов из других методов
        print(text)         
 
    def say_name(self):
        self._say(f"Hello, I am {self._name}")
 
    def say_how_old(self):
        self._say(f"I am {self._age} years old")

In [32]:
bob = Human("Bob", age=29)

bob.say_name()
bob.say_how_old()

Hello, I am Bob
I am 29 years old


Есть еще такая вещь как метод класса (classmethod). От обычного отличается тем, что обычному методу передается инстанс (self),
а методу класса - именно что класс (cls). По сути своей - просто декоратор внутри класса.

In [39]:
from datetime import date
def extract_description(user_string):
    return "открытие чемпионата мира по футболу"


def extract_date(user_string):
    return date(2018, 6, 14)

# в идеале тут конечно нормалтнаые функции которые что-то считают а не возвращают одно и то же

class Event:
    
    def __init__(self, description, event_date):
        self.description = description
        self.event_date = event_date
        
    def __str__(self):
        return f"Event \"{self.description}\" at {self.event_date}"
    
    @classmethod
    def from_string(cls, us_input):
        description = extract_description(us_input)
        event_date = extract_date(us_input)
        return cls(description,event_date)


In [40]:
event = Event.from_string("добавить в мой календарь открытие чемпионата мира по футболу на 14 июня 2018 года")
print(event)

Event "открытие чемпионата мира по футболу" at 2018-06-14


Так же есть такая вещь как статический метод класса (по сути своей - просто функция). Например, в Java функций нет вообще,
есть только методы и статические методы. Основное отличие статического метода от обычного заключается в том, что ему по умолчанию не передается ни объект, ни класс

In [42]:
class Human:
 
    def __init__(self, name, age=0):
        self.name = name
        self.age = age

    @staticmethod
    def is_age_valid(age):
        return 0 < age < 150

Как видно из примера выше, статик методу не передается ни self, ни cls.

Внутри классов есть еще один вид декораторов - property или вычисляемое свойство. У пропертей есть три основных атрибута - 
__setter__, __getter__ и __deleter__. Первый управляет присвоееним значения, второй - его извлечением и третий - удалением.

Предположим, у нас есть некий класс Robot с единственным атрибутом power.

In [44]:
class Robot:
    def __init__(self,power):
        self.power = power

Мы хотим, чтобы power был больше либо равен нулю, потому что отрицательная мощность - нонсенс. Очевидное решение:

In [45]:
class Robot:

    def __init__(self, power):
        self.power = power
    
    def set_power(self, power):
        if power < 0:
            self.power = 0
        else:
            self.power = power

Однако это решение не оптимальное. Если кто-то уже юзал наш код до появления метода set_power, и присвоил отрицательное значение, ему придется его переписывать, чего никому бы не хотелось. В такой ситуации мы можем использовать проперти:

In [4]:
class Robot:
 
    def __init__(self, power):
        self._power = power

    power = property()
    
    @power.setter
    def power(self,value):
        if value < 0:
            self._power = 0
        else:
            self._power = value
            
    @power.getter
    def power(self):
        return self._power
    
    @power.deleter
    def power(self):
        print("make robot useless")
        del self._power

In [6]:
wall_e = Robot(100)
wall_e.power = -20
print(wall_e.power)

0


Если нас не интересует setter и deleter, существует сокращенная форма записи:

In [ ]:
class Robot:
    def __init__(self, power):
        self._power = power
    
    @property
    def power(self):
        # здесь могут быть любые полезные вычисления
        return self._power